In [2]:
from models import proposed_model
from keras.optimizers import Adam
from keras.utils import np_utils
from callbacks import Step
import numpy as np
import random
import cv2
import os
import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt
import glob
import math
import tensorflow as tf
import random
import shap
import keras.backend as K


In [3]:
def process_batch(lines,img_path,inputH,inputW,train=True):
    imagew = 128
    imageh = 128
    num = len(lines)
    batch = np.zeros((num, imagew, imageh,1), dtype='float32')


    labels = np.zeros(num, dtype='int')
    for i in range(num):
        path = lines[i].split(' ')[0]
        label = lines[i].split(' ')[-1]

        label = label.strip('\n')
        label = int(label)

        img = path

        if train:
            #crop_x = random.randint(0, np.max([0, imagew-inputW]))
            #crop_y = random.randint(0, np.max([0, imageh-inputH]))
            #is_flip = random.randint(0, 1)

            try:
                image = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
            
            except Exception as e:
                print(img)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            #image = image[crop_y:crop_y + inputH, crop_x:crop_x + inputW]
            image = cv2.resize(image, (imagew, imageh))

            #if is_flip == 1:
            #    image = cv2.flip(image, 1)

            batch[i,:,:,0] = image
            labels[i] = label
        else:
            image = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
            #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


            #Hshmean = int(np.round(np.max([0, np.round((imageh-inputH)/2)])))
            #Wshmean = int(np.round(np.max([0, np.round((imagew-inputW)/2)])))
            #image = image[Hshmean:Hshmean+inputH,Wshmean:Wshmean+inputW]
            image = cv2.resize(image, (imagew, imageh))

            batch[i,:,:,0] = image
            labels[i] = label

    return batch, labels

In [47]:
folders = os.listdir('MIT-BIH_AD/')

In [48]:
folders

['.DS_Store', 'N', '.ipynb_checkpoints', 'B']

In [49]:
files = []
k=0
for w in folders:
    if w[0] == 'B' or w[0] == 'N':
        for i in os.listdir('MIT-BIH_AD/'+w):
            files.append('MIT-BIH_AD/'+w+'/'+i+' '+str(k))
            k=k+1


In [36]:
l = int(0.8*len(files))

In [37]:
random.shuffle(files)
train_list = files[:l]
val_list = files[l:]

In [38]:
len(val_list)

448

In [39]:
def generator_train_batch( t_list, batch_size, num_classes, img_path, inputH, inputW ):
    num = len(t_list)
    while True:
        new_line = []
        for m in range(num):
            new_line.append(t_list[m])

        for i in range(int(num/batch_size)):
            a = i*batch_size
            b = (i+1)*batch_size
            x_train, x_labels = process_batch(new_line[a:b], img_path, inputH, inputW, train=True)
            y = np_utils.to_categorical(np.array(x_labels), num_classes)
            yield x_train, y

In [40]:
def generator_val_batch(v_list,batch_size,num_classes,img_path,inputH,inputW):
    
    num = len(v_list)
    while True:
        new_line = []
        for m in range(num):
            new_line.append(v_list[m])
        for i in range(int(num / batch_size)):
            a = i * batch_size
            b = (i + 1) * batch_size
            y_test,y_labels = process_batch(v_list[a:b],img_path,inputH,inputW,train=False)
            y = np_utils.to_categorical(np.array(y_labels), num_classes)
            yield y_test, y

In [41]:
batch_size = 50
epochs = 1
input_h = 96
input_w = 96

In [42]:
num_classes = 2
k=10

In [43]:
model = proposed_model(nb_classes=num_classes)
lr = 0.001
adam = Adam(lr=lr)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 128, 128, 1)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 126, 126, 64)      640       
_________________________________________________________________
elu_17 (ELU)                 (None, 126, 126, 64)      0         
_________________________________________________________________
batch_normalization_17 (Batc (None, 126, 126, 64)      256       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 61, 61, 128)       73856     
_________________________________________________________________
elu_18 (ELU)                 (None, 61, 61, 128)       0   

In [44]:
history = model.fit_generator(
            generator_train_batch(train_list, batch_size, num_classes, '', input_h, input_w),
            steps_per_epoch=len(train_list) // batch_size,
            epochs=epochs,
            callbacks=[Step()],
            validation_data=generator_val_batch(val_list, batch_size, num_classes, '', input_h, input_w),
            validation_steps=len(val_list) // batch_size,
            verbose=1)

Epoch 1/1
35/35 [==============================] - 100s 3s/step - loss: 1.1641 - accuracy: 0.5051 - val_loss: 97.7460 - val_accuracy: 0.0000e+00


In [19]:
#model.save_weights('model_all')